In [ ]:
import sys
sys.path.append('/home/silja/fatepredict/fatepredict_experiments')

In [5]:
import sys
sys.path.append('/home/silja/fatepredict/fatepredict_experiments')

In [6]:
import zarr
import numpy as np
import networkx as nx
from skimage.measure import regionprops
#from segmentation import segment_stats
from linajea_cost_test import get_merge_graph_from_array
from linajea_cost_test import get_merge_graph
from funlib.math import decode64, encode64
import networkx as nx
from candidate_graph import iterate_tree

In [7]:
z = zarr.open('anno_alice_T2030_tiny.zarr','r')
gt_image = z['gt_trackimage'][:]
raw = z['Raw'][:]
fragments = z['Fragments'][:]


In [8]:
# annation label is 10000, 10001, 10004, 10005, 10007, 10010, 10013, 10016, 10017 pick them out
gt_mask = gt_image >=10000
gt_image = gt_mask * gt_image

In [9]:
# create annotation graph by gt_trackimage
time = 3
anno_graph = nx.DiGraph()


for label in np.unique(gt_image):
    if label == 0:
        continue   
    for t in range(time):
        mask = gt_image[t]==label
        regions = regionprops(mask.astype(int))
        for props in regions:
            z0, y0, x0 = props.centroid
            u = encode64((t, int(z0), int(y0), int(x0), int(props.area)),bits=[9,12,12,12,19])
        mask = gt_image[t]==label
        regions = regionprops(mask.astype(int))
        for props in regions:
            z0, y0, x0 = props.centroid
            v = encode64((t, int(z0), int(y0), int(x0), int(props.area)),bits=[9,12,12,12,19])
        anno_graph.add_edge(v, u, source = v, target = u)


In [6]:
# get the id list of one time frame
id, positions, _ = segment_stats(gt_image[0],0)

In [4]:
t = 0
merge_tree = z['Merge_tree/Merge/'+str(t)]
scores = z['Merge_tree/Scoring/'+str(t)]
merge_tree = get_merge_graph_from_array(merge_tree,scores)


In [10]:
# count the number of how many annotation id in merge tree
def count_anno_id(id,merge_tree):
    '''
    merge_tree: nx.Digrph merge_tree create by waterz 
    id: ndarray (n,) 1d array store all ids in GT image
    correct_id_num: int count the number of how many annotation node(id) in merge tree
    '''
    mt_id = list(merge_tree.nodes)
    correct_id_num = len(set(id)&set(mt_id))
    return correct_id_num 

correct_id_num = count_anno_id(id,merge_tree)

In [6]:
# get the id list of one time frame
id, positions, _ = segment_stats(gt_image[0],0)

<Labels layer 'fragments' at 0x7f639ff5c8b0>

In [ ]:
t = 0
merge_tree = z['Merge_tree/Merge/'+str(t)]
scores = z['Merge_tree/Scoring/'+str(t)]
merge_tree = get_merge_graph_from_array(merge_tree,scores)

# count the number of how many annotation id in merge tree
def count_anno_id(id,merge_tree):
    '''
    merge_tree: nx.Digrph merge_tree create by waterz 
    id: ndarray (n,) 1d array store all ids in GT image
    correct_id_num: int count the number of how many annotation node(id) in merge tree
    '''
    mt_id = list(merge_tree.nodes)
    correct_id_num = len(set(id)&set(mt_id))
    return correct_id_num 

correct_id_num = count_anno_id(id,merge_tree)


In [ ]:
import napari
viewer = napari.Viewer()
viewer.add_image(raw[3])
viewer.add_labels(gt_image)

In [ ]:
# read .gexf file which created by run_candidate_graph.ipynb
graph = nx.read_gexf('text.gexf')

In [ ]:
def get_select_graph(graph):
    select_graph = nx.DiGraph()
    for u,v,data in graph.edges(data=True):
        if data['selected']:
            select_graph.add_edge(v, u, source = v, target = u)
            select_graph.add_node(u, **graph.nodes[u])
            select_graph.add_node(v, **graph.nodes[v])
    return select_graph

# get the sovler output

In [10]:
# read .gexf file which created by run_candidate_graph.ipynb
graph = nx.read_graphml('test.graphml')

In [11]:
def get_select_graph(graph):
    select_graph = nx.DiGraph()
    for u,v,data in graph.edges(data=True):
        if data['selected']:
            select_graph.add_edge(v, u, source = v, target = u)
            select_graph.add_node(u, **graph.nodes[u])
            select_graph.add_node(v, **graph.nodes[v])
    return select_graph

select_graph = get_select_graph(graph)

In [12]:

for node in select_graph.nodes:
    if select_graph.out_degree(node) == 2:
        print(node)
    # find the devision node


1337143618181632
34234522901939712
5453646393247232
844545189216256
28569847583478785
879617892155393


In [13]:
def pick_select_edge(graph):
    edges = []
    selected_edges = []
    for u,v,data in graph.edges(data=True):
        edges.append((u,v))
        if data['selected']:
            selected_edges.append((u,v))
    return edges, selected_edges

edges, selected_edges = pick_select_edge(graph)

In [14]:
track_data = []
# creat a list for tracking node
track_id_node = []
track_id = 0
for node in select_graph.nodes:
    cor = decode64(int(node),dims=5,bits=[9,12,12,12,19])
    track_data.append([track_id,cor[0],cor[1],cor[2],cor[3]])
    track_id_node.append(node)
    track_id += 1

In [15]:
def select_nodes_t(select_graph,t):
    nodes_t = []
    for node in select_graph.nodes:
        cor = decode64(int(node),dims = 5,bits=[9,12,12,12,19])
        if cor[0]== t:
            nodes_t.append(node)
    return nodes_t
# pick out the start node

node_start = select_nodes_t(select_graph,0)
print('the ids of nodes in t0 are:', node_start)

the ids of nodes in t0 are: ['1020578718816256', '1794634908964352', '8584995398486528', '11892360728284160', '950218593931776', '13616583976948736', '387045272852992', '14425618334619136', '1125934287555072', '1337143618181632', '1020389761225728', '12420306700342784', '1724248995398144', '1337238096969728', '11751795078464512', '1126054584385536', '3131469249647616', '1372233480018432', '4433239475228160', '5770314342471680', '70600682897408', '9429446111005184', '27795748470920192', '352084251639808', '2005715371687936', '3870495678139392', '3307477059766272', '25860668156548096', '6333384552877056', '809352231393792', '950184223705600', '34234522901939712', '5453646393247232', '28323531230021120', '6825836952946176', '34199252666160128', '4538973003186688', '1161256111177728', '39582607616248320', '3061229369167872', '844545189216256', '10555337411133952', '2286992805068800', '1759450543162880', '7142659481143296', '8303520449037824', '1442567904297984', '809309334145024', '4222150

In [16]:
def find_path(graph, start, end):
    queue = [(start, [start])]
    while queue:
        (node, path) = queue.pop(0)
        for neighbor in graph.successors(node):
            if neighbor == end:
                yield path + [end]
            else:
                queue.append((neighbor, path + [neighbor]))
                
def find_all_paths(graph, start):
    paths = []
    for node in graph.nodes():
        paths.extend(list(find_path(graph, start, node)))
    return [path for i, path in enumerate(paths) if not any(path == subpath[:len(path)] for subpath in paths[:i] + paths[i+1:])]

def find_repeated_nodes(paths):
    node_counts = {}
    for path in paths:
        for node in path:
            node_counts[node] = node_counts.get(node, 0) + 1
    repeated_nodes = set([node for node, count in node_counts.items() if count > 1])
    return repeated_nodes

def pick_nodes_once(lst):
    output = []
    nodes = set()
    for item in lst:
        new_item = []
        for node in item:
            if node not in nodes:
                new_item.append(node)
                nodes.add(node)
        if new_item:
            output.append(new_item)
    return output

In [20]:
track_data = []
track_id = 0
label = 1
num_node = 0
cell_mask = np.zeros(fragments.shape)
#track_graph = nx.DiGraph()# graph for arboretum
track_graph = {}
for start in node_start:    
    paths = find_all_paths(select_graph, start)
    pick = pick_nodes_once(paths)
    main_track = label

    for path in pick:
        for node in path:
            num_node = num_node+1
            cor = decode64(int(node),dims=5,bits=[9,12,12,12,19])
            t = cor[0]
            track_data.append([label,cor[0],cor[1],cor[2],cor[3]])
            ids = z['Fragment_stats/id/'+str(t)]
            merge_tree = z['Merge_tree/Merge/'+str(t)]
            scores = z['Merge_tree/Scoring/'+str(t)]
            merge_tree = get_merge_graph_from_array(merge_tree,scores)

            if merge_tree.has_node(int(node)):
                sub_tree = iterate_tree(merge_tree,int(node))
                for a in sub_tree:
                    if a in list(ids):
                        # a is the leave node id
                        # get the label of this leave node
                        label_a = list(ids).index(a)+1
                        cell_mask[t][fragments[t] == label_a] = label
        if len(paths)>1 and main_track != label:
            #track_graph.add_edge(main_track,label)
            track_graph[label] = [main_track]
            track_graph[main_track] = [main_track]
        label = label+1
        

    if len(paths)>1:
    #print('******************')
        print('the paths are',paths)
        #print('******************')
        print('sub-set of paths:',pick)
        print('******************')


the paths are [['1337143618181632', '1161213163607553', '13335040276567554'], ['1337143618181632', '1513048298756097', '668606188754946']]
sub-set of paths: [['1337143618181632', '1161213163607553', '13335040276567554'], ['1513048298756097', '668606188754946']]
******************
the paths are [['34234522901939712', '211192140268033'], ['34234522901939712', '5629628389524481', '2005663836275714']]
sub-set of paths: [['34234522901939712', '211192140268033'], ['5629628389524481', '2005663836275714']]
******************
the paths are [['5453646393247232', '1759287323919361', '598168685248514'], ['5453646393247232', '1900050586272769', '422298364412418']]
sub-set of paths: [['5453646393247232', '1759287323919361', '598168685248514'], ['1900050586272769', '422298364412418']]
******************
the paths are [['844545189216256', '351963979973633', '3307459825371650'], ['844545189216256', '457508508402689', '5348127643009538']]
sub-set of paths: [['844545189216256', '351963979973633', '330745

In [21]:
track_graph

{11: [10],
 10: [10],
 34: [33],
 33: [33],
 36: [35],
 35: [35],
 45: [44],
 44: [44],
 58: [57],
 57: [57],
 70: [69],
 69: [69]}

In [22]:
track_data = np.array(track_data).astype(int)

In [23]:
import napari
viewer = napari.Viewer()
viewer.add_image(raw[3])
viewer.add_labels(gt_image)
viewer.add_labels(fragments)
tracks = napari.layers.Tracks(track_data,   
                              graph=track_graph, 
                              name="track_test")
viewer.add_labels(cell_mask.astype(int))
viewer.add_layer(tracks)
viewer.window.add_plugin_dock_widget(plugin_name="napari-arboretum", 
                                     widget_name="Arboretum")



(<napari._qt.widgets.qt_viewer_dock_widget.QtViewerDockWidget at 0x7f480a13ac20>,
 <napari_arboretum.plugin.Arboretum at 0x7f47fbf3eb00>)

In [25]:
from napari_arboretum import load_sample_data

track, segmentation = load_sample_data()
viewer = napari.Viewer()
viewer.add_layer(segmentation)
viewer.add_layer(track)
viewer.window.add_plugin_dock_widget(
    plugin_name="napari-arboretum", widget_name="Arboretum"
)

(<napari._qt.widgets.qt_viewer_dock_widget.QtViewerDockWidget at 0x7f4828db8160>,
 <napari_arboretum.plugin.Arboretum at 0x7f4808b12710>)

In [2]:
x = track.data

In [3]:
y = track.graph

In [26]:
y

{82: [53],
 83: [53],
 87: [82],
 88: [82],
 89: [68],
 90: [68],
 92: [35],
 93: [35],
 105: [52],
 106: [52],
 118: [5],
 119: [5],
 130: [24],
 131: [24],
 132: [33],
 133: [33],
 142: [14],
 143: [14],
 145: [144],
 146: [144],
 147: [44],
 148: [44],
 154: [137],
 155: [137],
 162: [22],
 163: [22],
 166: [120],
 167: [120],
 170: [36],
 171: [36],
 172: [34],
 173: [34],
 174: [8],
 175: [8],
 177: [26],
 178: [26],
 185: [140],
 186: [140],
 187: [57],
 188: [57],
 191: [9],
 192: [9],
 195: [39],
 196: [39],
 197: [185],
 198: [185],
 200: [13],
 201: [13],
 204: [30],
 205: [30],
 206: [93],
 207: [93],
 210: [119],
 211: [119],
 212: [41],
 213: [41],
 216: [11],
 217: [11],
 218: [38],
 220: [38],
 221: [50],
 222: [50],
 224: [21],
 225: [21],
 226: [29],
 227: [29],
 232: [43],
 233: [43],
 234: [45],
 236: [45],
 240: [12],
 241: [12],
 246: [56],
 248: [56],
 252: [27],
 253: [27],
 256: [92],
 257: [92],
 258: [58],
 259: [58],
 260: [19],
 261: [32],
 262: [32],
 263: 

In [1]:
import networkx as nx

g = nx.Graph()
g.add_edge(2,1)
g.add_edge(3,2)
g.add_edge(4,2)
g.add_edge(5,4)
g.add_edge(6,4)
g.add_edge(7,3)
g.add_edge(8,1)
g.add_edge(9,8)
g.add_edge(10,9)

# Perform depth-first search to obtain the desired subset
subset = []
visited = set()

def dfs(node):
    visited.add(node)
    subset.append(node)
    for neighbor in g.neighbors(node):
        if neighbor not in visited:
            dfs(neighbor)
            # Add the node again if it has multiple branches
            if len(list(g.neighbors(node))) > 1:
                subset.append(node)

dfs(1) # Start the search at node 1
subset

[1, 2, 3, 7, 3, 2, 4, 5, 4, 6, 4, 2, 1, 8, 9, 10, 9, 8, 1]

In [45]:
# example for track

'''
               /  5  
           /  4 - 6
      / 2 / - 3 - 7
    1/- 8 - - 9 - 10

'''

g = nx.DiGraph()
g.add_edge(2,1)
g.add_edge(8,1)

g.add_edge(3,2)
g.add_edge(4,2)

g.add_edge(5,4)
g.add_edge(6,4)

g.add_edge(7,3)

g.add_edge(9,8)
g.add_edge(10,9)

node_start=(1)
lst = [[1,2,4,5],[1,2,4,6],[1,2,3,7],[1,8,9,10]]

track_graph = {}
output = []
nodes = set()
for item in lst:
    new_item = []
    for node in item:
        if node not in nodes:
            nodes.add(node)
            new_item.append(node)
            if g.in_degree(node) == 2:
                output.append(new_item)
                new_item = []
                      
    if new_item:
        output.append(new_item)

for i in output:
    print(i)


[1]
[2]
[4]
[5]
[6]
[3, 7]
[8, 9, 10]


In [38]:
g.in_degree(4)

2

In [25]:
g.in_degree(node)

0

In [23]:
nodes

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [5]:
[[1],[2],[5],[3,7],[4],[6],[8,9,10]]

2

In [9]:
def find_sub_paths(g):
    sub_paths = []

    def dfs(v, path):
        if v != 1:
            sub_paths.append(path)
        for w in g.successors(v):
            if w not in path:
                dfs(w, path + [w])

    dfs(1, [1])
    return sub_paths

In [8]:
sub_paths = find_sub_paths(g)
print(sub_paths)

[]


In [30]:
subset = []
visited = set()
def dfs(node):
    visited.add(node)
    subset.append(node)
    for neighbor in g.neighbors(node):
        if neighbor not in visited:
            dfs(neighbor)
            # Add the node again if it has multiple branches
            if len(list(g.neighbors(node))) > 1:
                subset.append(node)

dfs(1) # Start the search at node 1
print(subset)
print(visited)

[1]
{1}


In [21]:
np.unique(cell)

array([  0, 173, 174])

In [43]:
select_graph.edges(data=True)

OutEdgeDataView([('1020578718816256', '2814981697443329', {'source': '1020578718816256', 'target': '2814981697443329'}), ('2814981697443329', '8374103897543682', {'source': '2814981697443329', 'target': '8374103897543682'}), ('1794634908964352', '2216847378224641', {'source': '1794634908964352', 'target': '2216847378224641'}), ('8584995398486528', '25966100969687553', {'source': '8584995398486528', 'target': '25966100969687553'}), ('25966100969687553', '70368752566786', {'source': '25966100969687553', 'target': '70368752566786'}), ('11892360728284160', '633361655664129', {'source': '11892360728284160', 'target': '633361655664129'}), ('633361655664129', '18436671112480770', {'source': '633361655664129', 'target': '18436671112480770'}), ('950218593931776', '8901869518722049', {'source': '950218593931776', 'target': '8901869518722049'}), ('8901869518722049', '950167060617730', {'source': '8901869518722049', 'target': '950167060617730'}), ('13616583976948736', '12736966088987137', {'source

ModuleNotFoundError: No module named 'btrack'